In [187]:
import pandas as pd
pd.options.display.max_colwidth = 1000
import os
import missingno as msno 
filelist = os.listdir('dataset') 
df_list = [pd.read_json('dataset/'+file, lines=True) for file in filelist]
df = pd.concat(df_list).reset_index(drop=True, inplace=False)

In [191]:
def fix_none(col_name):
    d = df.loc[(df[col_name].isna()) | (df[col_name].isnull()), ['url', col_name]].drop_duplicates()
    d = pd.DataFrame(d)
    
    urls = list(d['url'])
    
    elements = pd.DataFrame(df.loc[df['url'].isin(urls) & (df[col_name].isna() == False), ['url', col_name]].drop_duplicates())
    
    for index, row in categories.iterrows():
        df.loc[df['url'] == row['url'], col_name] = row[col_name]

In [192]:
fix_none('category')

In [194]:
fix_none('documentId')

KeyError: 'documentId'

In [193]:
df.loc[df['url'].isin(['http://adressa.no/meninger/kronikker/2017/02/13/pasientblikket-nedenfra-14203543.ece'])]

,eventId,category,activeTime,title,url,userId,publishtime,time,documentId
229,593997291,meninger|kronikker,NaN,"Pasientblikket nedenfra etableres der du ligger naken og sårbar, uten briller, klokke, telefon",http://adressa.no/meninger/kronikker/2017/02/13/pasientblikket-nedenfra-14203543.ece,cx:ibqugwavry9cac3r:1wuceknedj7u6,2017-02-13T08:22:00.000Z,1490052268,45720473a6f9e0ec49a8e787fcc142f0c09a386b
610182,902615920,meninger|kronikker,328.0,None,http://adressa.no/meninger/kronikker/2017/02/13/pasientblikket-nedenfra-14203543.ece,cx:i5nxvsulqgvf50sq:2jrht3akqsl90,None,1486974364,None
610197,1997033742,meninger|kronikker,NaN,None,http://adressa.no/meninger/kronikker/2017/02/13/pasientblikket-nedenfra-14203543.ece,cx:ib1vo01vq38f2mqc:20lut6o1pv35i,None,1486974390,None
610287,1405871248,meninger|kronikker,219.0,None,http://adressa.no/meninger/kronikker/2017/02/13/pasientblikket-nedenfra-14203543.ece,cx:iic1p9y54x6uprmt:3tg1zow5fw7nw,None,1486974554,None
610301,95902935,meninger|kronikker,NaN,None,http://adressa.no/meninger/kronikker/2017/02/13/pasientblikket-nedenfra-14203543.ece,cx:bqptsyehdx0s29n1fogr8r32l:1rbz8n7m7hyup,None,1486974580,None
...,...,...,...,...,...,...,...,...,...
636676,669529731,meninger|kronikker,NaN,"Pasientblikket nedenfra etableres der du ligger naken og sårbar, uten briller, klokke, telefon",http://adressa.no/meninger/kronikker/2017/02/13/pasientblikket-nedenfra-14203543.ece,cx:1md9tlyft7puk1sxf2y1n5jhh3:3iqvjco0i1l02,2017-02-13T08:22:00.000Z,1487058927,45720473a6f9e0ec49a8e787fcc142f0c09a386b
639903,1651547737,meninger|kronikker,124.0,"Pasientblikket nedenfra etableres der du ligger naken og sårbar, uten briller, klokke, telefon",http://adressa.no/meninger/kronikker/2017/02/13/pasientblikket-nedenfra-14203543.ece,cx:uan37jog4n8tbtzt1dy8ob95:p2el72fm2kwx,2017-02-13T08:22:00.000Z,1487065898,45720473a6f9e0ec49a8e787fcc142f0c09a386b
657923,131432829,meninger|kronikker,1.0,"Pasientblikket nedenfra etableres der du ligger naken og sårbar, uten briller, klokke, telefon",http://adressa.no/meninger/kronikker/2017/02/13/pasientblikket-nedenfra-14203543.ece,cx:ijq24khh55o2122b:289q18ivmeykj,2017-02-13T08:22:00.000Z,1487109010,45720473a6f9e0ec49a8e787fcc142f0c09a386b
1806649,634673048,meninger|kronikker,NaN,"Pasientblikket nedenfra etableres der du ligger naken og sårbar, uten briller, klokke, telefon",http://adressa.no/meninger/kronikker/2017/02/13/pasientblikket-nedenfra-14203543.ece,cx:ilqy7d6aigwerex2:2bpfg1d91lhlj,2017-02-13T08:22:00.000Z,1487582135,45720473a6f9e0ec49a8e787fcc142f0c09a386b


In [174]:
d = df.loc[(df['category'].isna()) | (df['category'].isnull()), ['url', 'category']].drop_duplicates()
pd.options.display.max_colwidth = 1000
d = pd.DataFrame(d)
urls = list(d['url'])
# urls
d
# SELET url WHERE category = NULL;

,url,category
0,http://adressa.no,None
16,http://adressa.no/vaeret/2017/03/20/vinterv%c3%a6r-igjen-midt-i-uken-betydelig-varmere-fra-helgen-14476360.ece,None
19,http://adressa.no/meninger/2017/03/10/ukultur-i-adressa-14427011.ece,None
22,http://adressa.no/nyheter/okonomi/2017/03/20/mye-%c3%a5-spare-p%c3%a5-%c3%a5-kj%c3%b8pe-apotekvarer-i-sverige-14478095.ece,None
31,http://adressa.no/pluss/magasin/2017/03/18/trondheimsrebusen-del-11-vet-du-dette-om-trondheim-14461510.ece,None
...,...,...
2205366,http://adressa.no/sport/2015/10/16/disse-to-tjente-suverent-mest-i-rbk-i-fjor-11691126.ece,None
2206355,http://adressa.no/digital/ny-samsung-telefon-tok-fyr-pa-fly-358b.html,None
2206905,http://adressa.no/100sport/ballsport/handballboom-blant-unge-gutter-pa-sorlandet-231760b.html,None
2207020,http://adressa.no/tema/tollvesenet,None


In [177]:
import itertools
categories = pd.DataFrame(df.loc[df['url'].isin(urls) & (df['category'].isna() == False), ['url', 'category', 'documentId']].drop_duplicates())
# indexes = list(categories.index)
# urls = list(urls)
# categories = list(itertools.chain(*categories.values.tolist()))
# for category, index, url in zip(categories, indexes, urls):
#     df.loc[df['url'] == url, 'category'] = categories.loc[index]
    
#for index, row in categories.iterrows():
#     df.loc[df['url'] == row['url'], 'category'] = row['category']

In [178]:
for index, row in categories.iterrows():
    df.loc[df['url'] == row['url'], 'category'] = row['category']

In [179]:
# http://adressa.no/nyheter/trondheim/2017/03/17/dette-er-%c2%abkystad-saken%c2%bb-i-korte-trekk-og-sakens-konsekvenser-14463144.ece

df.loc[df['url'].isin(['http://adressa.no/meninger/kronikker/2017/02/13/pasientblikket-nedenfra-14203543.ece'])]

,eventId,category,activeTime,title,url,userId,publishtime,time,documentId
229,593997291,meninger|kronikker,NaN,"Pasientblikket nedenfra etableres der du ligger naken og sårbar, uten briller, klokke, telefon",http://adressa.no/meninger/kronikker/2017/02/13/pasientblikket-nedenfra-14203543.ece,cx:ibqugwavry9cac3r:1wuceknedj7u6,2017-02-13T08:22:00.000Z,1490052268,45720473a6f9e0ec49a8e787fcc142f0c09a386b
610182,902615920,meninger|kronikker,328.0,None,http://adressa.no/meninger/kronikker/2017/02/13/pasientblikket-nedenfra-14203543.ece,cx:i5nxvsulqgvf50sq:2jrht3akqsl90,None,1486974364,None
610197,1997033742,meninger|kronikker,NaN,None,http://adressa.no/meninger/kronikker/2017/02/13/pasientblikket-nedenfra-14203543.ece,cx:ib1vo01vq38f2mqc:20lut6o1pv35i,None,1486974390,None
610287,1405871248,meninger|kronikker,219.0,None,http://adressa.no/meninger/kronikker/2017/02/13/pasientblikket-nedenfra-14203543.ece,cx:iic1p9y54x6uprmt:3tg1zow5fw7nw,None,1486974554,None
610301,95902935,meninger|kronikker,NaN,None,http://adressa.no/meninger/kronikker/2017/02/13/pasientblikket-nedenfra-14203543.ece,cx:bqptsyehdx0s29n1fogr8r32l:1rbz8n7m7hyup,None,1486974580,None
...,...,...,...,...,...,...,...,...,...
636676,669529731,meninger|kronikker,NaN,"Pasientblikket nedenfra etableres der du ligger naken og sårbar, uten briller, klokke, telefon",http://adressa.no/meninger/kronikker/2017/02/13/pasientblikket-nedenfra-14203543.ece,cx:1md9tlyft7puk1sxf2y1n5jhh3:3iqvjco0i1l02,2017-02-13T08:22:00.000Z,1487058927,45720473a6f9e0ec49a8e787fcc142f0c09a386b
639903,1651547737,meninger|kronikker,124.0,"Pasientblikket nedenfra etableres der du ligger naken og sårbar, uten briller, klokke, telefon",http://adressa.no/meninger/kronikker/2017/02/13/pasientblikket-nedenfra-14203543.ece,cx:uan37jog4n8tbtzt1dy8ob95:p2el72fm2kwx,2017-02-13T08:22:00.000Z,1487065898,45720473a6f9e0ec49a8e787fcc142f0c09a386b
657923,131432829,meninger|kronikker,1.0,"Pasientblikket nedenfra etableres der du ligger naken og sårbar, uten briller, klokke, telefon",http://adressa.no/meninger/kronikker/2017/02/13/pasientblikket-nedenfra-14203543.ece,cx:ijq24khh55o2122b:289q18ivmeykj,2017-02-13T08:22:00.000Z,1487109010,45720473a6f9e0ec49a8e787fcc142f0c09a386b
1806649,634673048,meninger|kronikker,NaN,"Pasientblikket nedenfra etableres der du ligger naken og sårbar, uten briller, klokke, telefon",http://adressa.no/meninger/kronikker/2017/02/13/pasientblikket-nedenfra-14203543.ece,cx:ilqy7d6aigwerex2:2bpfg1d91lhlj,2017-02-13T08:22:00.000Z,1487582135,45720473a6f9e0ec49a8e787fcc142f0c09a386b


In [78]:
df.loc[df['url'].isin(['http://adressa.no/nyheter/sortrondelag/2017/03/20/nektet-straffskyld-for-%c3%a5-ha-kvalt-kameraten-til-d%c3%b8de-14472170.ece']), 'category'] = categories.loc[102]
df[df['url'].isin(['http://adressa.no/nyheter/sortrondelag/2017/03/20/nektet-straffskyld-for-%c3%a5-ha-kvalt-kameraten-til-d%c3%b8de-14472170.ece'])]

,eventId,category,activeTime,title,url,userId,publishtime,time,documentId
102,1961153565,nyheter|sortrondelag,NaN,Nektet straffskyld for å ha kvalt kameraten til døde,http://adressa.no/nyheter/sortrondelag/2017/03/20/nektet-straffskyld-for-%c3%a5-ha-kvalt-kameraten-til-d%c3%b8de-14472170.ece,cx:13675964253951224092221:1wgstmuzo9vjl,2017-03-20T08:47:21.000Z,1490051217,cd18c3e23a1c35b3941c6d70ee0fe292928f82b9
1727,117292823,nyheter|sortrondelag,4.0,Nektet straffskyld for å ha kvalt kameraten til døde,http://adressa.no/nyheter/sortrondelag/2017/03/20/nektet-straffskyld-for-%c3%a5-ha-kvalt-kameraten-til-d%c3%b8de-14472170.ece,cx:i7t39cjvxfij5kmg:2fbipf7j5ztmt,2017-03-20T08:47:21.000Z,1490074463,cd18c3e23a1c35b3941c6d70ee0fe292928f82b9
3159,1664416892,nyheter|sortrondelag,NaN,Nektet straffskyld for å ha kvalt kameraten til døde,http://adressa.no/nyheter/sortrondelag/2017/03/20/nektet-straffskyld-for-%c3%a5-ha-kvalt-kameraten-til-d%c3%b8de-14472170.ece,cx:1tl7wkjzd8rhx3py1c7jssmy4i:jdnrvqwukk56,2017-03-20T08:47:21.000Z,1490078300,cd18c3e23a1c35b3941c6d70ee0fe292928f82b9
10291,791507213,nyheter|sortrondelag,1.0,Nektet straffskyld for å ha kvalt kameraten til døde,http://adressa.no/nyheter/sortrondelag/2017/03/20/nektet-straffskyld-for-%c3%a5-ha-kvalt-kameraten-til-d%c3%b8de-14472170.ece,cx:2epncctkr4u6h1bqea9kyb43qf:2jitd7ci10oin,2017-03-20T08:47:21.000Z,1490097837,cd18c3e23a1c35b3941c6d70ee0fe292928f82b9
10390,1906148918,nyheter|sortrondelag,74.0,Nektet straffskyld for å ha kvalt kameraten til døde,http://adressa.no/nyheter/sortrondelag/2017/03/20/nektet-straffskyld-for-%c3%a5-ha-kvalt-kameraten-til-d%c3%b8de-14472170.ece,cx:2h8vek54rt9dzfnr3o623bmkj:2z3p55mt8c11a,2017-03-20T08:47:21.000Z,1490098171,cd18c3e23a1c35b3941c6d70ee0fe292928f82b9
459296,1852528208,nyheter|sortrondelag,NaN,None,http://adressa.no/nyheter/sortrondelag/2017/03/20/nektet-straffskyld-for-%c3%a5-ha-kvalt-kameraten-til-d%c3%b8de-14472170.ece,cx:ib1vo01vq38f2mqc:20lut6o1pv35i,None,1490000191,None
459313,1157572747,nyheter|sortrondelag,NaN,None,http://adressa.no/nyheter/sortrondelag/2017/03/20/nektet-straffskyld-for-%c3%a5-ha-kvalt-kameraten-til-d%c3%b8de-14472170.ece,cx:ihx0v4mgarziegm3:2rlqndj9cmln7,None,1490000254,None
459360,1284205008,nyheter|sortrondelag,13.0,None,http://adressa.no/nyheter/sortrondelag/2017/03/20/nektet-straffskyld-for-%c3%a5-ha-kvalt-kameraten-til-d%c3%b8de-14472170.ece,cx:3lc90od7ivhp82gdltiqbi8ig3:1uinf7140u2er,None,1490000399,None
459377,148824761,nyheter|sortrondelag,39.0,None,http://adressa.no/nyheter/sortrondelag/2017/03/20/nektet-straffskyld-for-%c3%a5-ha-kvalt-kameraten-til-d%c3%b8de-14472170.ece,cx:2wdlftbi226e93j6ubrrr9ywlk:xi3bokxgktuc,None,1490000440,None
459378,976872756,nyheter|sortrondelag,NaN,None,http://adressa.no/nyheter/sortrondelag/2017/03/20/nektet-straffskyld-for-%c3%a5-ha-kvalt-kameraten-til-d%c3%b8de-14472170.ece,cx:iaxvnnrzl6wm1z5f:uf8xinqqiaa1,None,1490000441,None


In [183]:
df.loc[df['category'].isna(), 'url'].drop_duplicates()

0                                                                                                                   http://adressa.no
16                     http://adressa.no/vaeret/2017/03/20/vinterv%c3%a6r-igjen-midt-i-uken-betydelig-varmere-fra-helgen-14476360.ece
19                                                               http://adressa.no/meninger/2017/03/10/ukultur-i-adressa-14427011.ece
22         http://adressa.no/nyheter/okonomi/2017/03/20/mye-%c3%a5-spare-p%c3%a5-%c3%a5-kj%c3%b8pe-apotekvarer-i-sverige-14478095.ece
31                         http://adressa.no/pluss/magasin/2017/03/18/trondheimsrebusen-del-11-vet-du-dette-om-trondheim-14461510.ece
                                                                      ...                                                            
2205366                                    http://adressa.no/sport/2015/10/16/disse-to-tjente-suverent-mest-i-rbk-i-fjor-11691126.ece
2206355                                                       

In [184]:
df.loc[df['url'] == 'http://adressa.no/vaeret/2017/03/20/vinterv%c3%a6r-igjen-midt-i-uken-betydelig-varmere-fra-helgen-14476360.ece', 'category'].drop_duplicates()

16    None
Name: category, dtype: object